In [2]:
import torch
import torch.nn as nn        # Pytorch neural network package
import torch.optim as optim  # Pytorch optimization package
import torch.nn.functional as F
import numpy
import matplotlib.pyplot as plt

device = torch.device("cuda")

In [52]:
features = 16
# define a simple linear VAE #until now normal VAE without Beta
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
 
        # encoder 84*84 = 7’056
        self.enc0 = nn.Linear(in_features=84*84, out_features=1024)
        self.enc1 = nn.Linear(in_features=1024, out_features=512)
        self.enc2 = nn.Linear(in_features=512, out_features=features*2)
 
        # decoder 
        self.dec0 = nn.Linear(in_features=features, out_features=512)
        self.dec1 = nn.Linear(in_features=512, out_features=1024)
        self.dec2 = nn.Linear(in_features=1024, out_features=84*84)

    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    
    def encode(self, x):
        x = F.relu(self.enc0(x))
        x = F.relu(self.enc1(x))
        x = self.enc2(x)
        return x
    
    def decode(self, x):
        x = F.relu(self.dec0(x))
        x = F.relu(self.dec1(x))
        x = torch.sigmoid(self.dec2(x))
        return x

In [4]:
train_data = numpy.load('train_data100kFEB23.npy')
val_data = numpy.load('val_data20kFEB23.npy')

In [5]:
print(train_data.shape)

(101848, 84, 84)


In [53]:
BVAE = LinearVAE()
BVAE.load_state_dict(torch.load('C:/Users/erics/Documents/Programme/Bachelorarbeit/models/BVAE_Pong/B=10VAEMAR9'))
BVAE.to(device)

LinearVAE(
  (enc0): Linear(in_features=7056, out_features=1024, bias=True)
  (enc1): Linear(in_features=1024, out_features=512, bias=True)
  (enc2): Linear(in_features=512, out_features=32, bias=True)
  (dec0): Linear(in_features=16, out_features=512, bias=True)
  (dec1): Linear(in_features=512, out_features=1024, bias=True)
  (dec2): Linear(in_features=1024, out_features=7056, bias=True)
)

In [54]:
randomInput = torch.rand(256, 32, device=device) #32 = 2* features
print(randomInput)


tensor([[0.8187, 0.0032, 0.5362,  ..., 0.9796, 0.1716, 0.1676],
        [0.2054, 0.7165, 0.8497,  ..., 0.7378, 0.7991, 0.1649],
        [0.7359, 0.9039, 0.9010,  ..., 0.4380, 0.6387, 0.7754],
        ...,
        [0.2164, 0.7388, 0.6928,  ..., 0.3644, 0.7257, 0.3567],
        [0.1468, 0.4003, 0.8375,  ..., 0.1828, 0.6380, 0.0214],
        [0.8535, 0.0142, 0.9714,  ..., 0.5682, 0.7995, 0.2638]],
       device='cuda:0')


In [55]:
identMatrix = torch.eye(32, device = device)
print(identMatrix)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')


In [15]:
def reparameterize(mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample

In [56]:
x = identMatrix.view(-1, 2, 16) # features = 64
mu = x[:, 0, :] # the first feature values as mean
log_var = x[:, 1, :] # the other feature values as variance

Input = reparameterize(mu, log_var)
print(Input)

tensor([[-4.1383e-01, -3.5095e-01, -5.4483e-01,  6.4840e-01, -1.2507e+00,
         -6.7929e-01,  2.3495e+00,  1.2239e+00,  2.6287e-01,  6.4396e-01,
         -1.2033e+00, -1.9343e+00, -7.4019e-01, -2.2000e-02,  5.1638e-01,
         -2.2708e+00],
        [ 1.0807e+00,  3.0531e-01,  7.4953e-01, -1.3234e+00,  4.3975e-01,
          2.2519e+00, -9.6164e-01, -4.2214e-01,  1.0107e+00, -8.0612e-01,
          1.0159e+00, -3.7896e-01,  9.3369e-01,  7.8460e-01, -6.2665e-01,
         -1.0970e+00],
        [-2.4618e-01,  4.7421e-01,  4.9739e-01,  5.8420e-01, -1.4450e+00,
          9.9763e-01,  4.0780e-01, -9.0324e-02,  7.5607e-01,  7.8983e-01,
         -5.8380e-02,  4.7372e-01, -1.1854e-02, -2.7450e-01,  1.0655e+00,
          1.7108e-01],
        [-5.3228e-01, -1.3410e+00,  3.7653e-01, -4.1147e-02, -1.1549e-02,
         -6.5181e-02,  1.3281e+00, -1.2299e+00, -6.6659e-01, -7.2607e-01,
          1.0154e+00,  9.3163e-01,  1.8341e+00, -1.0154e+00,  9.6072e-01,
         -4.1892e-01],
        [-2.8865e+00

In [57]:
recon = BVAE.decode(Input).view(-1, 84, 84)
print(recon)

tensor([[[0.3304, 0.3349, 0.3322,  ..., 0.3302, 0.3324, 0.3320],
         [0.3389, 0.3358, 0.3285,  ..., 0.3245, 0.3226, 0.3270],
         [0.3254, 0.3293, 0.3324,  ..., 0.3213, 0.3264, 0.3318],
         ...,
         [0.3231, 0.3259, 0.3302,  ..., 0.3127, 0.3090, 0.3350],
         [0.3329, 0.3390, 0.3319,  ..., 0.3326, 0.3192, 0.3314],
         [0.5633, 0.5620, 0.5654,  ..., 0.5544, 0.5545, 0.5663]],

        [[0.3394, 0.3361, 0.3324,  ..., 0.3427, 0.3472, 0.3487],
         [0.3413, 0.3401, 0.3459,  ..., 0.3535, 0.4262, 0.3370],
         [0.3346, 0.3386, 0.3336,  ..., 0.3585, 0.4158, 0.3399],
         ...,
         [0.3336, 0.3373, 0.3460,  ..., 0.3425, 0.3389, 0.3375],
         [0.3376, 0.3423, 0.3413,  ..., 0.3301, 0.3223, 0.3456],
         [0.5588, 0.5628, 0.5541,  ..., 0.5592, 0.5570, 0.5640]],

        [[0.3456, 0.3457, 0.3465,  ..., 0.3466, 0.3449, 0.3489],
         [0.3488, 0.3414, 0.3472,  ..., 0.3472, 0.3490, 0.3410],
         [0.3436, 0.3458, 0.3426,  ..., 0.3352, 0.3476, 0.

In [58]:
recon = recon.detach().cpu().numpy()

In [59]:
print(recon.shape)

(32, 84, 84)


In [65]:
axes=[]
fig = plt.figure(figsize=(200,200))

for a in range(recon.shape[0]):
    axes.append( fig.add_subplot(4, 8, a+1) )
    axes[-1].set_title(str(a+1))  
    plt.imshow(recon[a])
    

plt.show()